In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class Discriminator(nn.Module):
  def __init__(self,in_features):
    super(Discriminator,self).__init__()
    self.in_features = in_features
    self.disc = nn.Sequential(
        
         nn.Linear(self.in_features,128),
         nn.LeakyReLU(0.1),
         nn.Linear(128,1),
         nn.Sigmoid(),
        
    )

  def forward(self,x):

    return self.disc(x)
  
class Generator(nn.Module):
  def __init__(self,z_dim,in_dim):
    super(Generator,self).__init__()
    self.z_dim = z_dim
    self.in_dim = in_dim
    self.gen = nn.Sequential(
        
         nn.Linear(z_dim,256),
         nn.LeakyReLU(0.1),
         nn.Linear(256,in_dim),
         nn.Tanh()
        
    )

  def forward(self,x):
    return self.gen(x)

lr = 3e-4
z_dim = 64
image_dim = 28*28*1
batch_size = 64
num_epochs = 3

disc = Discriminator(in_features = image_dim).to(device)
gen = Generator(z_dim,image_dim).to(device)
fixed_noise = torch.randn((batch_size,z_dim)).to(device)
transforms = transforms.Compose(
    [
     transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))
    ]
)
dataset = datasets.MNIST(root='./data',download=True,transform=transforms)
loader = DataLoader(dataset = dataset,batch_size = batch_size,shuffle=True)
opt_disc = optim.Adam(disc.parameters(),lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/fake")
writer_real = SummaryWriter(f"runs/real")
step = 0

cuda


In [ ]:
for epoch in tqdm(range(num_epochs)):
  for batch_idx,(real,_) in enumerate(loader):
      real = real.view(-1,784).to(device)
      batch_size = real.shape[0]

      ## Loss of Discriminator max log(D(real))+ (1-D(G(noise)))
      noise = torch.randn((batch_size,z_dim )).to(device)
      fake = gen(noise)
      disc_real = disc(real).view(-1)
      lossD_real = criterion(disc_real,torch.ones_like(disc_real))
      disc_fake = disc(fake).view(-1)
      lossD_fake = criterion(disc_fake,torch.zeros_like(disc_fake))
      lossD = (lossD_real + lossD_fake) /2
      disc.zero_grad()
      lossD.backward(retain_graph=True)
      opt_disc.step()

      ## Loss of Generator min(log(1-D(G(z))))
      output = disc(fake).view(-1)
      lossG = criterion(output, torch.ones_like(output))
      gen.zero_grad()
      lossG.backward()
      opt_gen.step()
      if batch_idx == 0:
          print(
              f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                    Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
          )

          with torch.no_grad():
              fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
              data = real.reshape(-1, 1, 28, 28)
              img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
              img_grid_real = torchvision.utils.make_grid(data, normalize=True)

              writer_fake.add_image(
                  "Mnist Fake Images", img_grid_fake, global_step=step
              )
              writer_real.add_image(
                  "Mnist Real Images", img_grid_real, global_step=step
              )
              step += 1

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch [0/3] Batch 0/938                     Loss D: 0.7468, loss G: 0.6545
Epoch [1/3] Batch 0/938                     Loss D: 0.4492, loss G: 1.1230
Epoch [2/3] Batch 0/938                     Loss D: 0.2853, loss G: 1.6457


In [ ]:
# writer_fake.close()
# writer_real.close()

In [ ]:
!pip install torch torchvision

In [ ]:
 !tensorboard --logdir='/content/runs'

2021-10-01 08:16:44.804562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-01 08:16:44.813268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-01 08:16:44.813957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL

In [ ]:
!tensorboard dev upload --logdir /content/runs/ \
--name "My latest experiment"\
# --description "Simple comparison of several hyperparameters"

2021-10-01 08:17:00.688576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-01 08:17:00.697658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-01 08:17:00.698509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/HM8KLLR5QHOU6WYGyzQWxg/

[2021-10-01T08:17:00] Started scanning logdir.


Interrupted.
Traceback (most recent call last):
  File "/usr/lo